In [1]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

Repository: 'deb https://ppa.launchpadcontent.net/jonathonf/ffmpeg-4/ubuntu/ jammy main'
Description:
Backport of FFmpeg 4 and associated libraries. Now includes AOM/AV1 support!

FDK AAC is not compatible with GPL and FFmpeg can't be redistributed with it included. Please don't ask for it to be added to this public PPA.

---

PPA supporters:

BigBlueButton (https://bigbluebutton.org)

---

Donate to FFMPEG: https://ffmpeg.org/donations.html
Donate to Debian: https://www.debian.org/donations
Donate to this PPA: https://ko-fi.com/jonathonf
More info: https://launchpad.net/~jonathonf/+archive/ubuntu/ffmpeg-4
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/jonathonf-ubuntu-ffmpeg-4-jammy.list
Adding key to /etc/apt/trusted.gpg.d/jonathonf-ubuntu-ffmpeg-4.gpg with fingerprint 4AB0F789CBA31744CC7DA76A8CF63AD3F06FC659
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http:/

In [2]:
# Step 0: Setup in Kaggle Notebook librosa datasets transformers evaluate
!pip install torchaudio  soundfile gdown pandas chardet num2words jiwer tqdm 
import gdown
import os
import pandas as pd
import torchaudio
from datasets import Dataset, DatasetDict
import librosa
import soundfile as sf

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 91.6 kB/s eta 0:00:00 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=f260c1ab6dacad2d71079b531e2311b12ad644dca834c0679ea8bba10c7dd8c1
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


In [3]:
!pip install pyloudnorm

In [4]:
import os
import chardet

transcript_folder = "/kaggle/input/edi-tamil-age/Training-20250321T063337Z-001/Training/Transcripts"
output_folder = "/kaggle/working/converted_transcripts"

os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(transcript_folder):
    if file.endswith(".txt"):
        file_path = os.path.join(transcript_folder, file)
        output_path = os.path.join(output_folder, file)

        # Read raw bytes to detect encoding
        with open(file_path, "rb") as f:
            raw_data = f.read()
        
        detected_encoding = chardet.detect(raw_data)["encoding"]

        # Read using detected encoding and save as UTF-8 in the writable directory
        with open(file_path, "r", encoding=detected_encoding, errors="replace") as f:
            text = f.read()
        
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(text)

print("All transcript files converted to UTF-8 and saved in:", output_folder)


All transcript files converted to UTF-8 and saved in: /kaggle/working/converted_transcripts


In [5]:
import os
import pandas as pd
from tqdm import tqdm
wav_path = '/kaggle/input/edi-tamil-age/Training-20250321T063337Z-001/Training/Audio'
transcript_path = '/kaggle/working/converted_transcripts'
wav_files = [f for f in os.listdir(wav_path) if f.endswith(".wav")]
transcript_files = [f for f in os.listdir(transcript_path) if f.endswith(".txt")]


data = {
    'audio':[],
    'transcript':[]
}
for file in tqdm(wav_files,desc='Processing'):
    name = os.path.splitext(file)[0]
    data['audio'].append(file)
    with open(f'{transcript_path}/{name}.txt', 'r', encoding='utf-8') as f:
        data['transcript'].append(f.read().strip())


df = pd.DataFrame(data)

Processing: 100%|██████████| 908/908 [00:00<00:00, 35246.90it/s]


In [6]:
df.sample(10)

,audio,transcript
25,Audio - 8_16.wav,மேடம் இந்த ஸ்கூல் ல பையன சேய் வந்திருக்கீங்களா...
622,Audio - 22_29.wav,என விலை போடுறிங்க கதிரிக்கலாம் பீட்ரூட் கேரட் ...
315,Audio - 33_11.wav,நம்ம கோயில்ல டோக்கன் எடுக்கனுமா\nடோக்கன் எடுத்...
138,Audio - 25_04.wav,புதுசா டி பாஸ் கட்டணும் எங்க போய் வாங்கணும் டி...
47,Audio - 17_26.wav,"காய்கள் வேற உருளைக்கிழங்கு, கருணைக்கிழங்கு வேற..."
876,Audio - 29_43.wav,அந்த பீரட்டு எவ்வளவு கொடுப்பீங்க. பால் பீரட் வ...
48,Audio - 29_24.wav,இது கோயம்புத்தூர் வழியா போகுங்களா இல்ல திருப்ப...
411,Audio - 1_15.wav,உள்ள மருந்த கொடுங்க இந்த மாத்திர என்ன விலை இந்...
361,Audio - 11_25.wav,கம்மியான ரேட்ல நல்லா செய்ங்க \nமெடிக்கல்ல தலை ...
802,Audio - 29_10.wav,அடுத்து வந்து எனக்கு அடிக்கடி லோன் தேவைப்படும்...


In [7]:
import os
import librosa
import numpy as np
import soundfile as sf
import pyloudnorm
from pyloudnorm import Meter
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")

def preprocess_audio(audio_path):
    waveform, sr = librosa.load(audio_path, sr=16000, mono=True)
    if len(waveform) < 16000 * 0.4:
        return None
    meter = Meter(sr)
    loudness = meter.integrated_loudness(waveform)
    target_loudness = -16
    waveform = pyloudnorm.normalize.loudness(waveform, loudness, target_loudness)
    waveform = waveform - np.mean(waveform)
    waveform = np.clip(waveform, -1.0, 1.0)
    output_path = f"/kaggle/working/processed_audio/{os.path.basename(audio_path)}"
    sf.write(output_path, waveform, 16000, subtype='PCM_16')
    return output_path

def process_all_audio(df):
    os.makedirs('/kaggle/working/processed_audio', exist_ok=True)
    processed_paths = []
    for x in tqdm(df['audio'], desc="Processing audio"):
        processed_path = preprocess_audio(f"{wav_path}/{x}")
        if processed_path:
            processed_paths.append(processed_path)
        else:
            processed_paths.append(None)
    df['audio'] = processed_paths
    return df

df = process_all_audio(df)

Processing audio: 100%|██████████| 908/908 [00:57<00:00, 15.69it/s]


In [8]:
import re
import unicodedata
import pandas as pd
from tqdm import tqdm

def normalize_tamil_text(df):
    processed_texts = []

    for _, row in tqdm(df.iterrows(), desc='Processing transcripts', total=len(df)):
        text = row['transcript']
        text = re.sub(r'<\|.*?\|>', '', text)
        text = text.replace('\u200c', '')
        text = unicodedata.normalize('NFC', text)
        text = re.sub(r'\s+', ' ', text).strip()
        tamil_punctuation = '''!()-“”…•<>?@[]^_`{|}~'''
        translator = str.maketrans('', '', tamil_punctuation)
        text = text.translate(translator)
        processed_texts.append(text)

    df['transcript'] = processed_texts
    df = df[df['transcript'].str.strip().astype(bool)]
    return df


df = normalize_tamil_text(df)

df = df.dropna(subset=['transcript'])

Processing transcripts: 100%|██████████| 908/908 [00:00<00:00, 9939.01it/s]


In [9]:
df.shape

(908, 2)

In [10]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df[['audio', 'transcript']])

In [11]:
from datasets import Dataset

# Shuffle and split
split_dataset = train_dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [12]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from jiwer import wer, cer
import bitsandbytes as bnb
from datasets import load_dataset

processor = Wav2Vec2Processor.from_pretrained("anuragshas/wav2vec2-xlsr-53-tamil")
model = Wav2Vec2ForCTC.from_pretrained("anuragshas/wav2vec2-xlsr-53-tamil", use_safetensors=True)

2025-05-10 08:34:36.725681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746866076.919893      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746866076.974456      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/542 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [13]:
from datasets import Audio

# Set audio column to automatically decode audio
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
eval_dataset = eval_dataset.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    # Process audio
    audio = batch["audio"]
    inputs = processor(audio["array"], sampling_rate=16000, return_tensors="pt")
    batch["input_values"] = inputs.input_values[0]
    batch["attention_mask"] = inputs.attention_mask[0]

    # Tokenize transcript
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcript"]).input_ids
    return batch

train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(prepare_dataset, remove_columns=eval_dataset.column_names)


Map:   0%|          | 0/726 [00:00<?, ? examples/s]

Map:   0%|          | 0/182 [00:00<?, ? examples/s]

In [14]:
import evaluate
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = pred_logits.argmax(-1)

    pred_str = processor.batch_decode(pred_ids)
    label_ids = pred.label_ids
    label_str = processor.batch_decode(label_ids, group_tokens=False)

    return {
        "wer": wer_metric.compute(predictions=pred_str, references=label_str),
        "cer": cer_metric.compute(predictions=pred_str, references=label_str)
    }


In [15]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": f["input_values"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt"
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.tokenizer.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt"
            )

        # Replace padding with -100 to ignore in loss
        labels = labels_batch["input_ids"].masked_fill(labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100)

        batch["labels"] = labels
        return batch


In [16]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
from jiwer import wer, cer
import bitsandbytes as bnb
from datasets import load_dataset


def compute_metrics(pred):
    pred_ids = pred.predictions.argmax(-1)
    labels = pred.label_ids
    pred_text = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_text = processor.batch_decode(labels, skip_special_tokens=True)
    wer_score = wer(label_text, pred_text)
    cer_score = cer(label_text, pred_text)
    return {"wer": wer_score, "cer": cer_score}

training_args = TrainingArguments(
    output_dir="./xlsr_large_tamil",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    report_to="none",
    gradient_accumulation_steps=2,
    save_steps=500,
    eval_steps=500,
)
data_collator = DataCollatorCTCWithPadding(processor=processor)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    data_collator=data_collator,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Wer,Cer
1,2.372000,1.906741,0.976103,0.495122
2,2.052600,1.851203,0.967729,0.461912
3,1.944900,1.790971,0.953227,0.452263
4,1.891200,1.729735,0.940359,0.451863
5,1.837000,1.730075,0.941381,0.443627
6,1.815300,1.727353,0.942606,0.442028


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 158230848 vs 158230736

In [17]:
# Save the model and processor (tokenizer and feature extractor)
model_save_path = "./xlsr_large_tamil_finetuned"
processor_save_path = "./xlsr_large_tamil_finetuned_processor"

# Save the model
model.save_pretrained(model_save_path)

# Save the processor (tokenizer and feature extractor)
processor.save_pretrained(processor_save_path)

OSError: [Errno 28] No space left on device: './xlsr_large_tamil_finetuned'

In [19]:
!pip install bitsandbytes

In [23]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig
import torch

# 4-bit quantization configuration (NF4)
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load processor and model
processor_ = Wav2Vec2Processor.from_pretrained("anuragshas/wav2vec2-xlsr-53-tamil")
model_ = Wav2Vec2ForCTC.from_pretrained(
    "anuragshas/wav2vec2-xlsr-53-tamil",
    quantization_config=nf4_config,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    ctc_zero_infinity=True,
    diversity_loss_weight=100,
    use_safetensors=True,
)

# Freeze feature extractor layers (this will freeze the convolutional layers in the model)
model_.freeze_feature_extractor()

# Apply LoRA to the model (attention layers like q_proj and v_proj)
lora_config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05, 
    bias="none"
)

model_ = get_peft_model(model, lora_config)
model_.print_trainable_parameters()  # This will show how many parameters are trainable

# Prepare the model for k-bit training (this adjusts the model for quantized weight training)
model_ = prepare_model_for_kbit_training(model)

# Setup for fine-tuning with Trainer (assuming datasets and other parameters are ready)
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./xlsr_large_tamil_",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    save_total_limit=2,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    report_to="none",
    gradient_accumulation_steps=2,
    save_steps=500,
    eval_steps=500,
)
data_collator = DataCollatorCTCWithPadding(processor=processor)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    data_collator=data_collator,
)

trainer.train()


NotImplementedError: 